# 2 - Знакомство с устройством рекуррентных нейронных сетей

Взято из официальной документации [PyTorch](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html).

Оригинальный автор: [Sean Robertson](https://github.com/spro/practical-pytorch).

![](assets/unrolled_rnn.png)
*Источник: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)*

В этом туториале мы построим с нуля рекуррентную нейронную сеть. Наша сеть будет считывать поочередно последовательность символов слова и в конце предсказывать вероятности происхождения той или иной фамилии.

In [ ]:
import glob
import os

import unicodedata
import string
import random

import torch
import torch.nn as nn
from tqdm.notebook import tqdm

## Подготовка данных

Выведем список наших файлов:

In [ ]:
def findFiles(path):
    return glob.glob(path)

print(findFiles('data/surnames/*.txt'))

Напишем функцию для перевода всех имен из Юникода в ASCII символы:

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Прочитываем все файлы с именами, и добавляем в список, соответствующий определенной категории по названию файла:

In [ ]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

category_to_lines = {}

for filename in findFiles('data/surnames/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    lines = readLines(filename)
    category_to_lines[category] = lines

In [ ]:
len(category_to_lines)

In [ ]:
idx_to_label = list(category_to_lines.keys())
idx_to_label

In [ ]:
label_to_idx = {label: idx for idx, label in enumerate(idx_to_label)}
label_to_idx

In [ ]:
n_categories = len(idx_to_label)

Посмотрим на примеры имен:

In [ ]:
for label in idx_to_label:
    print(f'{label}: {category_to_lines[label][:5]}')
    print('--------------------')

## Превращаем строки в Тензоры

Каждый символ имени мы представим как "one-hot вектор" размера ``<1 x n_letters>``. Вектор весь будет заполнен нулями за исключением одной единицы на месте, соответсвующему определенному символу. Например, символ `b` будет представлен следующим образом:
``<0 1 0 0 0 ... 0 0>``.

Тогда каждое слово мы будем представлять в виде 2D матрицы размера ``<line_length x 1 x n_letters>``.

Мы будем обрабатывать по одному имени за раз, но так как PyTorch оперирует батчами добавляем одно дополнительное измерение.

In [ ]:
# Найти индекс символа в all_letters, например, "a" = 0, "e" = 5
def letterToIndex(letter):
    return all_letters.find(letter)

# Конвертировать один символ в тензор <1 x n_letters>
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Конвертирвоать строку тензор <line_length x 1 x n_letters>,
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [ ]:
print(letterToTensor('J'))

In [ ]:
print(lineToTensor('Jones').size())

## Строим модель

Построение собственной модели обычно состоит из реализации класса наследующего ``nn.Module``. В новом классе должны быть реализованы два обязательных метода:
- ``__init__`` - описание всех слоев нейронной сети;
- ``forward`` - все шаги, которая должна пройти сеть от момента получения входных значений до момента возврата предсказаний.

In [ ]:
print(nn.Module.__doc__)

На каждом шаге на вход модели будет подаваться один закодированный символ и скрытое состояние модели полученное при обработке предыдущего символа. Если на вход модели подается первый символ, то вместо предыдущего скрытого состояния подается тензор из нулей.

У модели внутри будет два линейных слоя:
- `i2h` будет возвращать новое скрытое состояние
- `i2o` будет возвращать предсказание сети

На вход оба линейных слоя будут принимать объединение входного вектора и предыдущего скрытого состояния.

Также в собственную модель не надо добавлять ``nn.Softmax(dim=1)``, так как на этапе тестирования и предсказания вероятность не нужна, а [реализация](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) функции ошибки ``nn.CrossEntropyLoss`` уже включает функцию Softmax.

![](assets/rnn_from_scratch.png)


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

Попробуем запустить модель на одном символе:

In [ ]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

На выходе получаем тензор ``<1 x n_categories>``, где наибольшее значение будет около предсказываемой категории.

## Обучение модели
### Подготовка к обучению
Несколько дополнительных функций. Первая функция -- для того, чтобы перевести вывод из нейронной сети в лейбл категории. Используем ``Tensor.topk`` для получения индекса элемента с наибольшим значением.

In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return idx_to_label[category_i], category_i

print(categoryFromOutput(output))

Также нужна функция для выбора случайного обучающего примера из набора данных:

In [ ]:
import random

def randomChoice(l):
    idx = random.randint(0, len(l) - 1)
    return idx, l[idx]

def randomTrainingExample():
    category_idx, category = randomChoice(idx_to_label)
    _, line = randomChoice(category_to_lines[category])
    category_tensor = torch.tensor([label_to_idx[category]], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

### Обучение модели

Подготавливаем функцию ошибки:

In [ ]:
criterion = nn.CrossEntropyLoss()

На каждом шаге обучения:

-  Создаем входной и целевой тензор
-  Создаем скрытое состояние из нулей
-  Обрабатываем поочередно каждый символ

   -  И обновляем скрытое состояние рекуррентной нейронной сети

-  Сравниваем последний выход сети с целевым тензором
-  Вычисляем градиенты
-  Возвращаем вывод модели и значение ошибки

In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # К значениям каждого параметра прибавляем его градиент умноженный на шаг обучения
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Храним значения функции ошибки, чтобы нарисовать график
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Выводим номер итерации, значение ошибки, имя и ответ модели
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Среднее значение лосса добавляем в список
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

Рисуем график изменения значения ошибки




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

## Оцениваем результаты

Построим матрицу ошибки:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

In [ ]:
predictions = []
labels = []

for label, names in category_to_lines.items():
    for name in names:
        category_tensor = torch.tensor([label_to_idx[label]], dtype=torch.long)
        line_tensor = lineToTensor(name)

        output = evaluate(line_tensor)
        guess, guess_i = categoryFromOutput(output)
        predictions.append(guess)
        labels.append(label)

In [ ]:
len(predictions), len(labels)

In [ ]:
cm = confusion_matrix(predictions, labels, labels=idx_to_label, normalize='true') # , normalize='pred'

In [ ]:
cmd_obj = ConfusionMatrixDisplay(cm, display_labels=idx_to_label)
cmd_obj.plot(include_values=False, xticks_rotation='vertical')

## Обрабатываем пользовательский ввод




In [ ]:
from torch.nn.functional import softmax

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))
        output = softmax(output, dim=1)

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, idx_to_label[category_index]))
            predictions.append([value, idx_to_label[category_index]])

In [ ]:
predict('Manning')
predict('Jurafsky')
predict('Braslavski')
predict('Efimov')
predict('Schuetze')
predict('Nikolenko')
predict('Rogers')